In [1]:
import httpx
import json
import openai
import os
import requests
from typing import List
from dotenv import load_dotenv
import time

load_dotenv(".env")
print(os.environ["CANONICAL_CACHE_API_KEY"])
print(os.environ["CANONICAL_CACHE_HOST"])

7z3Ugy1R.dT5sLE2cbmAnTf41SybUEABmbninWIRZ
http://localhost:8001/


In [2]:
TEMPERATURE = 0
CACHECLIENT = openai.OpenAI(
    base_url=os.environ["CANONICAL_CACHE_HOST"],
    api_key="canbeanything",
    http_client=httpx.Client(
        headers={
            "X-Canonical-Api-Key": os.environ["CANONICAL_CACHE_API_KEY"],
        },
    ),
)

OPENAICLIENT = openai.OpenAI(
    api_key= os.environ["OPENAI_API_KEY"],
)

In [7]:
def complete(
    client: openai.OpenAI,
    messages: List[dict[str:str]],
    stream: bool,
    temperature: int = 0,
) -> openai.ChatCompletion:
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        stream=stream,
        temperature=temperature,
        messages=messages,
    )


def displaycompletion(completion: openai.ChatCompletion, stream: bool) -> str:
    msg = ""
    if stream:
        for chunk in completion:
            if chunk.choices[0].delta.content is not None:
                chunk = chunk.choices[0].delta.content
                msg += chunk
                print(chunk, end="", flush=True)
    else:
        msg = completion.choices[0].message.content
        print(msg)
    return msg


def updatecache(messages: List[dict[str:str]], temp: int = 0) -> None:
    requests.request(
        method="POST",
        url=f"{os.environ['CANONICAL_CACHE_HOST']}api/v1/cache",
        headers={
            "Content-Type": "application/json",
            "X-Canonical-Api-Key": os.environ["CANONICAL_CACHE_API_KEY"],
        },
        data=json.dumps(
            {
                "messages": messages,
                "temperature": temp,
                "model": "gpt-3.5-turbo",
            }
        ),
    )


async def ask_canonical(message: str) -> None:
    global chat_history
    # Add user's message to the conversation history
    chat_history.append({"role": "user", "content": message})

    stream = False
    try:
        print('RETRIEVING FROM CACHE...')
        print(chat_history)

        tic = time.perf_counter()
        response = complete(CACHECLIENT, chat_history, stream, TEMPERATURE)
        toc = time.perf_counter()
        print(f"Retrieved cached response in {toc - tic:0.2f} seconds.")
        print("\nCached response:", response)

        msg = displaycompletion(response, stream)
        chat_history.append({"role": "assistant", "content": msg})

    except openai.NotFoundError as e:
        pass
        print('REQUESTING API RESPONSE...')
        print(chat_history)

        stream = True
        tic = time.perf_counter()
        response = complete(OPENAICLIENT, chat_history, stream, TEMPERATURE)
        toc = time.perf_counter()
        print(f"Retrieved API response in {toc - tic:0.2f} seconds.")

        msg = displaycompletion(response, stream)
        print('Completion response:', msg)

        chat_history.append({"role": "assistant", "content": msg})
        updatecache(chat_history, TEMPERATURE)


#### User A making repetitive requests

In [8]:
global chat_history
chat_history = [
    {"role": "system", "content": "You are a customer support assistant for company ACME. Talk to a customer regarding his or her inquiries and concerns about ACME's products and services. ACME sells home electronics through their online store."},
    {"role": "assistant", "content": "Hello, how can I help you today?"}
]

In [9]:
# First request
message = "I'd like to ask for a refund."
await ask_canonical(message)

RETRIEVING FROM CACHE...
[{'role': 'system', 'content': "You are a customer support assistant for company ACME. Talk to a customer regarding his or her inquiries and concerns about ACME's products and services. ACME sells home electronics through their online store."}, {'role': 'assistant', 'content': 'Hello, how can I help you today?'}, {'role': 'user', 'content': "I'd like to ask for a refund."}]
Retrieved cached response in 0.06 seconds.

Cached response: ChatCompletion(id='chatcmpl-3wF7SbdzY92a4GjNQpJTCS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry to hear that you're looking for a refund. Can you please provide me with your order number and the reason for the refund so I can assist you further?", role='assistant', function_call=None, tool_calls=None))], created=1710189788, model='gpt-3.5-turbo', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=0, prompt_tokens=0, total_tok

In [10]:
# Identical request #2
message = "I'd like to ask for a refund."
await ask_canonical(message)

RETRIEVING FROM CACHE...
[{'role': 'system', 'content': "You are a customer support assistant for company ACME. Talk to a customer regarding his or her inquiries and concerns about ACME's products and services. ACME sells home electronics through their online store."}, {'role': 'assistant', 'content': 'Hello, how can I help you today?'}, {'role': 'user', 'content': "I'd like to ask for a refund."}, {'role': 'assistant', 'content': "I'm sorry to hear that you're looking for a refund. Can you please provide me with your order number and the reason for the refund so I can assist you further?"}, {'role': 'user', 'content': "I'd like to ask for a refund."}]
REQUESTING API RESPONSE...
[{'role': 'system', 'content': "You are a customer support assistant for company ACME. Talk to a customer regarding his or her inquiries and concerns about ACME's products and services. ACME sells home electronics through their online store."}, {'role': 'assistant', 'content': 'Hello, how can I help you today?'

In [11]:
# Identical request #3
message = "I'd like to ask for a refund."
await ask_canonical(message)

RETRIEVING FROM CACHE...
[{'role': 'system', 'content': "You are a customer support assistant for company ACME. Talk to a customer regarding his or her inquiries and concerns about ACME's products and services. ACME sells home electronics through their online store."}, {'role': 'assistant', 'content': 'Hello, how can I help you today?'}, {'role': 'user', 'content': "I'd like to ask for a refund."}, {'role': 'assistant', 'content': "I'm sorry to hear that you're looking for a refund. Can you please provide me with your order number and the reason for the refund so I can assist you further?"}, {'role': 'user', 'content': "I'd like to ask for a refund."}, {'role': 'assistant', 'content': "Of course, I'd be happy to assist you with that. Can you please provide me with your order number and the reason for the refund so I can start the process for you?"}, {'role': 'user', 'content': "I'd like to ask for a refund."}]
REQUESTING API RESPONSE...
[{'role': 'system', 'content': "You are a custome

#### Another User B making the same request

In [12]:
# Simulate another user asking the same request
chat_history = [
    {"role": "system", "content": "You are a customer support assistant for company ACME. Talk to a customer regarding his or her inquiries and concerns about ACME's products and services. ACME sells home electronics through their online store."},
    {"role": "assistant", "content": "Hello, how can I help you today?"}
]
message = "I'd like to ask for a refund."
await ask_canonical(message)

RETRIEVING FROM CACHE...
[{'role': 'system', 'content': "You are a customer support assistant for company ACME. Talk to a customer regarding his or her inquiries and concerns about ACME's products and services. ACME sells home electronics through their online store."}, {'role': 'assistant', 'content': 'Hello, how can I help you today?'}, {'role': 'user', 'content': "I'd like to ask for a refund."}]
Retrieved cached response in 0.05 seconds.

Cached response: ChatCompletion(id='chatcmpl-W7NLA3qFCvouSp6mvDbJhq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry to hear that you're looking for a refund. Can you please provide me with your order number and the reason for the refund so I can assist you further?", role='assistant', function_call=None, tool_calls=None))], created=1710189815, model='gpt-3.5-turbo', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=0, prompt_tokens=0, total_tok